In [108]:
import pandas as pd

In [109]:
melb_data_ps =  pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_df = melb_data_ps.copy()

melb_df.head()

,index,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,...,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Coordinates
0,0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,...,1,202.0,126.0,1970,Yarra,-37.7996,144.9984,Northern Metropolitan,4019,"-37.7996, 144.9984"
1,1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,...,0,156.0,79.0,1900,Yarra,-37.8079,144.9934,Northern Metropolitan,4019,"-37.8079, 144.9934"
2,2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,...,0,134.0,150.0,1900,Yarra,-37.8093,144.9944,Northern Metropolitan,4019,"-37.8093, 144.9944"
3,3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,...,1,94.0,126.0,1970,Yarra,-37.7969,144.9969,Northern Metropolitan,4019,"-37.7969, 144.9969"
4,4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,...,2,120.0,142.0,2014,Yarra,-37.8072,144.9941,Northern Metropolitan,4019,"-37.8072, 144.9941"


In [110]:
# список уникальных значений в столбце
melb_df['Address'].unique()

array(['85 Turner St', '25 Bloomburg St', '5 Charles St', ...,
       '83 Power St', '96 Verdon St', '6 Agnes St'],
      shape=(13378,), dtype=object)

In [111]:
# количество уникальных значений в столбце
print(melb_df['Address'].nunique())

13378


Если мы прогнозируем цену объекта, то такое большое количество возможных категорий может плохо сказаться на модели, которую мы бы хотели в дальнейшем построить на наших данных. Говорят, что такой признак, скорее всего, не имеет статистической значимости, потому что не позволяет разделить данные на группы, которые можно сравнить по целевому признаку.
> Из-за таких признаков зависимость между целевым признаком, который мы хотим предсказать, и признаками, на основе которых мы делаем предсказание, становится очень сложной. При этом точность моделирования при учёте такого признака может не повыситься, а даже снизиться, а производительность однозначно резко упадёт.

In [112]:
print(melb_df['Address'][177])
print(melb_df['Address'].iloc[1812]) # так как столбец индекс = индексу DataFrame, то iLoc = Loc
print(melb_df['Address'].loc[9001])

2/119 Railway St N
9/400 Dandenong Rd
172 Danks St


In [113]:
# извлечение подтипа улицы (St, Cn, Rd и т.д.), убирает географическую метку при помощи написанной функции и применение этой функции в методе apply()
test = pd.Series(['2/119 Railway St N', '9/400 Dandenong Rd', '172 Danks St'])


def get_street_type(addres):
    exclude_list = ['N', 'S', 'W', 'E']
    addres_list = addres.split()
    street_type = addres_list[-1]
    if street_type in exclude_list:
        street_type = addres_list[-2]
    
    return street_type

# street_type = melb_df['Address'].apply(get_street_type)
# print(street_type)

print(test.apply(get_street_type))

0    St
1    Rd
2    St
dtype: object


> Обратите внимание, что функция пишется для одного элемента столбца, а метод apply() применяется к каждому его элементу. Используемая функция обязательно должна иметь возвращаемое значение.

In [114]:
street_type.nunique()

56

In [115]:
street_type.value_counts()

Address
St           8012
Rd           2825
Ct            612
Dr            447
Av            321
Gr            311
Pde           211
Pl            169
Cr            152
Cl            100
La             67
Bvd            53
Tce            47
Wy             40
Avenue         40
Cct            25
Hwy            24
Parade         15
Boulevard      13
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Gdns            4
Grn             4
Fairway         4
Mews            4
Crossway        3
Righi           3
Victoria        2
Ridge           2
Crofts          2
Esp             2
Glade           1
Gra             1
Ave             1
Woodland        1
Outlook         1
Hts             1
Highway         1
Athol           1
Summit          1
Grand           1
Res             1
Nook            1
Eyrie           1
Dell            1
East            1
Loop            1
Grange          1
Terrace         1
Cove            1
Qy              1
Co

- к результату метода __value_counts__ применим метод __nlargest()__, который возвращает n наибольших значений из Series. Зададим n=10, т. е. мы хотим отобрать десять наиболее популярных подтипов. Извлечём их названия с помощью атрибута __index__, а результат занесём в переменную __popular_stypes__:

In [116]:
street_type.value_counts().nlargest(n=10)

Address
St     8012
Rd     2825
Ct      612
Dr      447
Av      321
Gr      311
Pde     211
Pl      169
Cr      152
Cl      100
Name: count, dtype: int64

In [117]:
popular_types = street_type.value_counts().nlargest(n=10).index
print(popular_types)

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object', name='Address')


- Теперь, когда у нас есть список наиболее популярных подтипов улиц __popular_types__, введём __lambda-функцию__, которая будет проверять, есть ли строка __x__ в этом перечне (__popular_types__), и, если __True__, __lambda-функция__ будет возвращать __x__, в противном случае она будет возвращать строку __'other'__. Наконец, применим такую функцию к Series __street_types__, полученной ранее, через метод __apply()__, а результат определим в новый столбец таблицы __StreetType__

In [118]:
melb_df['StreetType'] = street_type.apply(lambda x: x if x in popular_types else 'other')
melb_df['StreetType']

0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

In [119]:
melb_df['StreetType'].nunique()

11

In [120]:
melb_df['StreetType'].unique()

array(['St', 'other', 'Rd', 'Gr', 'Ct', 'Dr', 'Pde', 'Pl', 'Cl', 'Cr',
       'Av'], dtype=object)

- Теперь, у нас нет потребности хранить признак Address, так как, если конкретное местоположение объекта всё же и влияет на его стоимость, то оно определяется столбцами Longitude и Lattitude. Удалим его из нашей таблицы:

In [121]:
melb_df = melb_df.drop('Address', axis=1)


In [122]:
melb_df

,index,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Coordinates,StreetType
0,0,Abbotsford,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067,...,202.0,126.0,1970,Yarra,-37.79960,144.99840,Northern Metropolitan,4019,"-37.7996, 144.9984",St
1,1,Abbotsford,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067,...,156.0,79.0,1900,Yarra,-37.80790,144.99340,Northern Metropolitan,4019,"-37.8079, 144.9934",St
2,2,Abbotsford,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067,...,134.0,150.0,1900,Yarra,-37.80930,144.99440,Northern Metropolitan,4019,"-37.8093, 144.9944",St
3,3,Abbotsford,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067,...,94.0,126.0,1970,Yarra,-37.79690,144.99690,Northern Metropolitan,4019,"-37.7969, 144.9969",other
4,4,Abbotsford,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067,...,120.0,142.0,2014,Yarra,-37.80720,144.99410,Northern Metropolitan,4019,"-37.8072, 144.9941",St
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13575,13575,Wheelers Hill,4,h,1245000.0,S,Barry,26/08/2017,16.7,3150,...,652.0,126.0,1981,NaN,-37.90562,145.16761,South-Eastern Metropolitan,7392,"-37.90562, 145.16761",Cr
13576,13576,Williamstown,3,h,1031000.0,SP,Williams,26/08/2017,6.8,3016,...,333.0,133.0,1995,NaN,-37.85927,144.87904,Western Metropolitan,6380,"-37.85927, 144.87904",Dr
13577,13577,Williamstown,3,h,1170000.0,S,Raine,26/08/2017,6.8,3016,...,436.0,126.0,1997,NaN,-37.85274,144.88738,Western Metropolitan,6380,"-37.85274, 144.88738",St
13578,13578,Williamstown,4,h,2500000.0,PI,Sweeney,26/08/2017,6.8,3016,...,866.0,157.0,1920,NaN,-37.85908,144.89299,Western Metropolitan,6380,"-37.85908, 144.89299",St


# ___РЕКОМЕНДАЦИЯ___

## по уменьшению числа уникальных значений в признаке, который описывается категориями:
1. Определите (хотя бы на глаз) соотношение числа уникальных категорий интересующего вас признака к общему числу объектов в таблице. Если это соотношение превышает значение 30 %, то это уже повод задуматься над уменьшением числа категорий и перейти к шагу 2.
2. Если ваш признак уникален для каждого объекта, например адрес, имя или название, то такой признак, скорее всего, не имеет статистической значимости. От таких признаков чаще всего избавляются. Однако можно попробовать выделить из этого признака какие-то общие черты, например, как мы это сделали с подтипами улиц. Такой же трюк можно произвести, например, с названиями компаний, в которых может быть скрыт признак типа организации (из строки «ООО Три Слепые Мыши» можно извлечь ООО — общество с ограниченной ответственностью).
3. Если даже после преобразования число уникальных категорий всё ещё велико, можно попробовать с помощью метода value_counts() оценить, есть ли в данных категории, которые употребляются гораздо реже, чем остальные. Если такие категории присутствуют, переходите к шагу 4.
4. Можно подобрать число  популярных категорий таким образом, чтобы эти категории покрывали большую часть ваших данных.  
Когда вы выбрали оптимальное число, переходите к шагу 5.
5. Наконец, можно совершить преобразование, обозначив категории, не попавшие в число популярных, как «другие».  
>Такая методика является очень популярной и хорошо показывает себя на практике. Однако не нужно ей злоупотреблять: применяя эту методику ко всем столбцам подряд, вы рискуете потерять немалую долю полезной информации из ваших данных. Внимательно изучите интересующий вас признак, прежде чем преобразовывать его.

# Самостоятельная работа

In [126]:

import pandas as pd
test_series_1 = pd.Series([
    'Опыт работы 8 лет 3 месяца',
    'Опыт работы 3 года 5 месяцев',
    'Опыт работы 1 год 9 месяцев',
    'Опыт работы 3 месяца',
    'Опыт работы 6 лет'
])

test_series_2 = pd.Series([
    'Опыт работы 5 лет',
    'Опыт работы 5 месяцев',
    'Опыт работы 1 год 1 месяц',
    'Опыт работы 3 месяца',
    'Опыт работы 7 лет'
])

def get_experience(arg):
    year_exp = ['лет', 'год']
    month_exp = 'месяц'
    experion_list = arg.split()
    
    if (experion_list[-3][:3] in year_exp) and (experion_list[-1][:5] == month_exp):
        return int(experion_list[-2]) + int(experion_list[-4]) * 12
    
    elif experion_list[-1][:5] == month_exp:
        return int(experion_list[-2])
                 
    elif experion_list[-1][:3] in year_exp:
        return int(experion_list[-2]) * 12

print(test_series_1)
print(test_series_1.apply(get_experience))
print(test_series_2)
print(test_series_2.apply(get_experience))


0      Опыт работы 8 лет 3 месяца
1    Опыт работы 3 года 5 месяцев
2     Опыт работы 1 год 9 месяцев
3            Опыт работы 3 месяца
4               Опыт работы 6 лет
dtype: object
0    99
1    41
2    21
3     3
4    72
dtype: int64
0            Опыт работы 5 лет
1        Опыт работы 5 месяцев
2    Опыт работы 1 год 1 месяц
3         Опыт работы 3 месяца
4            Опыт работы 7 лет
dtype: object
0    60
1     5
2    13
3     3
4    84
dtype: int64


In [124]:
unique_list = []
for i_col in melb_df.columns:
    item = (i_col, melb_df[i_col].nunique(), melb_df[i_col].dtypes)
    unique_list.append(item)
unique_df = pd.DataFrame(unique_list,
                         columns=['Column_Name', 'Num_Unique', 'Type']
                         ).sort_values('Num_Unique', ignore_index=True)

display(unique_df)

,Column_Name,Num_Unique,Type
0,Type,3,object
1,Method,5,object
2,Regionname,8,object
3,Bathroom,9,int64
4,Rooms,9,int64
5,Car,11,int64
6,StreetType,11,object
7,Bedroom,12,int64
8,CouncilArea,33,object
9,Date,58,object
